In [2]:
from torchvision.datasets import *
import torchvision
import scipy

import torch, time, os, copy, random, imageio, os, shutil
import torch.nn as nn
import numpy as np
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
import os

from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

data_transforms = {
    'train_rgb_wide': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomRotation(15,),
        transforms.RandomCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    
    'train_rgb_inc': transforms.Compose([
        transforms.Resize((342,342)),
        transforms.RandomRotation(15,),
        transforms.RandomCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    
    'train_gray': transforms.Compose([
        transforms.Resize((120,120)),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomRotation(15,),
        transforms.RandomCrop(120),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])]),

    
    'test_rgb_wide': transforms.Compose([
        transforms.Resize((244,244)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
        
    'test_rgb_inc': transforms.Compose([
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
        
    'test_gray': transforms.Compose([
        transforms.Resize((120,120)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])]) }

def load_data(data, batch_size=64):
    return DataLoader(data, batch_size=batch_size, shuffle=True)


In [4]:
class CustomConcatDataset(Dataset):
    def __init__(self, first_dataset, second_dataset, first_class):
        self.first_dataset = first_dataset
        self.second_dataset = second_dataset
        self.classes = first_class
        self.first_dataset_length = len(first_dataset)
        self.second_dataset_length = len(second_dataset)

    def __getitem__(self, index):
        if index < self.first_dataset_length:
            # If the index is within the range of the first dataset, get the sample from it
            img, label = self.first_dataset[index]
        else:
            # If the index is beyond the first dataset, get the sample from the second dataset
            img, label = self.second_dataset[index - self.first_dataset_length]
            label += self.classes
            # Add the number of classes in the first dataset to the label

        return img, label

    def __len__(self):
        # The length of the concatenated dataset is the sum of the lengths of the individual datasets
        return self.first_dataset_length + self.second_dataset_length


In [5]:
def train_model(model, criterion, optimizer, scheduler, data_loader, dataset_sizes, num_epochs):
    since = time.time()

    best_acc = 0.0
    test_token=0

    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            '''
            Test when a better validation result is found
            '''

            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model

In [6]:
stl_class = 10
flower_class = 102
aircraft_class = 100
kmi_class = 10
emi_bal_class = 814
emi_byc_class = 255

# kmi dataset is the background info of emni dataset
back_rgb_wide = ImageFolder("/kaggle/input/back-datas/back", transform=data_transforms['train_rgb_wide'])
back_rgb_inc = ImageFolder("/kaggle/input/back-datas/back", transform=data_transforms['train_rgb_inc'])
back_gray_kmi = ImageFolder("/kaggle/input/back-datas/emi_bal", transform=data_transforms['train_gray'])
back_gray_emi = ImageFolder("/kaggle/input/back-datas/kmi", transform=data_transforms['train_gray'])

In [7]:
stl_train = STL10("./data", split="train", download=True, transform = data_transforms['train_rgb_wide'])
stl_test = STL10("./data", split="train", download=True, transform = data_transforms['test_rgb_wide'])

100%|██████████| 2640397119/2640397119 [06:44<00:00, 6520027.08it/s] 


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [8]:
flower_train = Flowers102(root= "./data", split='train', download=True, transform=data_transforms['train_rgb_wide'])
flower_test = Flowers102(root= "./data", split='test', download=True, transform=data_transforms['test_rgb_wide'])

100%|██████████| 344862509/344862509 [00:01<00:00, 200018972.41it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 275054.29it/s]


100%|██████████| 14989/14989 [00:00<00:00, 7859535.27it/s]


In [16]:
airdata_train = FGVCAircraft("./data", split="train", download=True, transform = data_transforms['train_rgb_inc'])
airdata_test = FGVCAircraft("./data", split="test", download=True, transform = data_transforms['test_rgb_inc'])

100%|██████████| 2753340328/2753340328 [00:23<00:00, 116440085.18it/s]


Extracting ./data/fgvc-aircraft-2013b.tar.gz to ./data


In [20]:
kmi_train = KMNIST(root= "./data", train=True, download=True, transform=data_transforms['train_gray'])
kmi_test = KMNIST(root= "./data", train=False, download=True, transform=data_transforms['test_gray'])

100%|██████████| 18165135/18165135 [00:29<00:00, 618571.56it/s] 


Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



100%|██████████| 29497/29497 [00:00<00:00, 133051.05it/s]


Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



100%|██████████| 3041136/3041136 [00:04<00:00, 622643.64it/s]


Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



100%|██████████| 5120/5120 [00:00<00:00, 10115325.71it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



In [7]:
emi_bal_train = EMNIST("./data", split="balanced", train=True, download=True, transform = data_transforms['train_gray'])
emi_bal_test = EMNIST("./data", split="balanced", train=False, download=True, transform = data_transforms['test_gray'])

emi_byc_train = EMNIST("./data", split="byclass", train=True, download=True, transform = data_transforms['train_gray'])
emi_byc_test = EMNIST("./data", split="byclass", train=False, download=True, transform = data_transforms['test_gray'])

100%|██████████| 561753746/561753746 [00:03<00:00, 160672599.58it/s]


Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw


In [8]:
def prepare_data(Data1_train, Data1_test, DataClass, batch_size, back_info, Data2=None):
    if not back_info:
        data_loader_dict = {'train': load_data(Data1_train, batch_size), 'test': load_data(Data1_test, batch_size)}
        data_size = {'train': len(Data1_train), 'test': len(Data1_test)}
    else:
        data_with_backinfo = CustomConcatDataset(Data1_train, Data2, DataClass)
        data_loader_dict = {'train': load_data(data_with_backinfo, batch_size), 'test': load_data(Data1_test, batch_size)}
        data_size = {'train': len(data_with_backinfo), 'test': len(Data1_test)}
        
    return data_loader_dict, data_size

In [9]:
def start_train(model_pre, DataClass, DataTrain, DataTest, model_name, BackData=None, back_info = False, lrate=0.001, batch_size=32, epochs=15):
    model_ft = model_pre
    num_ftrs = model_ft.fc.in_features
    
    if back_info:
        model_ft.fc = nn.Linear(num_ftrs, DataClass+1)
    else:
        model_ft.fc = nn.Linear(num_ftrs, DataClass)
    
    if model_name == "inception":
        model_ft.aux_logits=False
    elif model_name == "kmni" or model_name == "emni":
        model_ft.conv1 = nn.Conv2d(1, 64, (7,7), stride=(2,2), padding=(3,3), bias=False)

    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=lrate, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    if lrate == 0.1:
        exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
    else:
        exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=1)
    
    data_loader_dict, data_size = prepare_data(DataTrain, DataTest, DataClass, batch_size, back_info, BackData)

    model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, data_loader_dict, data_size, epochs)
    return model_ft

In [11]:
model = start_train(models.wide_resnet101_2(pretrained=True), stl_class, stl_train, stl_test, "wide", epochs=20)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:00<00:00, 281MB/s]  


Epoch 0/19
----------
train Loss: 0.7611 Acc: 0.8360
test Loss: 0.0846 Acc: 0.9892

Epoch 1/19
----------
train Loss: 0.1277 Acc: 0.9690
test Loss: 0.0335 Acc: 0.9934

Epoch 2/19
----------
train Loss: 0.0851 Acc: 0.9788
test Loss: 0.0222 Acc: 0.9954

Epoch 3/19
----------
train Loss: 0.0609 Acc: 0.9832
test Loss: 0.0151 Acc: 0.9972

Epoch 4/19
----------
train Loss: 0.0395 Acc: 0.9924
test Loss: 0.0066 Acc: 0.9996

Epoch 5/19
----------
train Loss: 0.0299 Acc: 0.9932
test Loss: 0.0042 Acc: 0.9996

Epoch 6/19
----------
train Loss: 0.0319 Acc: 0.9906
test Loss: 0.0046 Acc: 0.9992

Epoch 7/19
----------
train Loss: 0.0237 Acc: 0.9954
test Loss: 0.0025 Acc: 1.0000

Epoch 8/19
----------
train Loss: 0.0294 Acc: 0.9914
test Loss: 0.0028 Acc: 0.9998

Epoch 9/19
----------
train Loss: 0.0133 Acc: 0.9974
test Loss: 0.0018 Acc: 0.9998

Epoch 10/19
----------
train Loss: 0.0149 Acc: 0.9972
test Loss: 0.0015 Acc: 1.0000

Epoch 11/19
----------
train Loss: 0.0147 Acc: 0.9976
test Loss: 0.0015 Acc

In [13]:
model = start_train(models.wide_resnet101_2(pretrained=True), stl_class, stl_train, stl_test, "wide", BackData=back_rgb_wide, back_info = True, epochs=20)

Epoch 0/19
----------
train Loss: 0.5760 Acc: 0.8604
test Loss: 0.0888 Acc: 0.9802

Epoch 1/19
----------
train Loss: 0.0932 Acc: 0.9762
test Loss: 0.0300 Acc: 0.9928

Epoch 2/19
----------
train Loss: 0.0542 Acc: 0.9868
test Loss: 0.0133 Acc: 0.9978

Epoch 3/19
----------
train Loss: 0.0380 Acc: 0.9910
test Loss: 0.0085 Acc: 0.9994

Epoch 4/19
----------
train Loss: 0.0318 Acc: 0.9928
test Loss: 0.0058 Acc: 0.9996

Epoch 5/19
----------
train Loss: 0.0238 Acc: 0.9943
test Loss: 0.0046 Acc: 0.9998

Epoch 6/19
----------
train Loss: 0.0156 Acc: 0.9970
test Loss: 0.0029 Acc: 1.0000

Epoch 7/19
----------
train Loss: 0.0149 Acc: 0.9973
test Loss: 0.0021 Acc: 1.0000

Epoch 8/19
----------
train Loss: 0.0105 Acc: 0.9980
test Loss: 0.0018 Acc: 1.0000

Epoch 9/19
----------
train Loss: 0.0138 Acc: 0.9961
test Loss: 0.0017 Acc: 1.0000

Epoch 10/19
----------
train Loss: 0.0117 Acc: 0.9970
test Loss: 0.0013 Acc: 1.0000

Epoch 11/19
----------
train Loss: 0.0084 Acc: 0.9981
test Loss: 0.0010 Acc

In [19]:
model = start_train(models.wide_resnet101_2(pretrained=True), flower_class, flower_train, flower_test, "wide", lrate=0.001, epochs=20)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/19
----------
train Loss: 4.6382 Acc: 0.0137
test Loss: 4.5325 Acc: 0.0483

Epoch 1/19
----------
train Loss: 4.3999 Acc: 0.1451
test Loss: 4.3270 Acc: 0.1981

Epoch 2/19
----------
train Loss: 4.0875 Acc: 0.4108
test Loss: 4.0261 Acc: 0.3532

Epoch 3/19
----------
train Loss: 3.6835 Acc: 0.5598
test Loss: 3.6587 Acc: 0.4667

Epoch 4/19
----------
train Loss: 3.2229 Acc: 0.6637
test Loss: 3.3205 Acc: 0.5363

Epoch 5/19
----------
train Loss: 2.7807 Acc: 0.7559
test Loss: 2.9532 Acc: 0.5824

Epoch 6/19
----------
train Loss: 2.3850 Acc: 0.7784
test Loss: 2.5879 Acc: 0.6261

Epoch 7/19
----------
train Loss: 2.0234 Acc: 0.8039
test Loss: 2.2631 Acc: 0.6796

Epoch 8/19
----------
train Loss: 1.6896 Acc: 0.8647
test Loss: 1.9590 Acc: 0.7219

Epoch 9/19
----------
train Loss: 1.3846 Acc: 0.9010
test Loss: 1.6999 Acc: 0.7536

Epoch 10/19
----------
train Loss: 1.1490 Acc: 0.9225
test Loss: 1.4878 Acc: 0.7769

Epoch 11/19
----------
train Loss: 0.9169 Acc: 0.9422
test Loss: 1.3023 Acc

In [20]:
model = start_train(models.wide_resnet101_2(pretrained=True), flower_class, flower_train, flower_test, "wide", BackData=back_rgb_wide, back_info=True)

Epoch 0/14
----------
train Loss: 1.3921 Acc: 0.7233
test Loss: 4.6326 Acc: 0.0000

Epoch 1/14
----------
train Loss: 1.2056 Acc: 0.7401
test Loss: 4.6141 Acc: 0.0015

Epoch 2/14
----------
train Loss: 1.1953 Acc: 0.7419
test Loss: 4.5601 Acc: 0.0122

Epoch 3/14
----------
train Loss: 1.1723 Acc: 0.7437
test Loss: 4.3849 Acc: 0.0402

Epoch 4/14
----------
train Loss: 1.1134 Acc: 0.7549
test Loss: 4.0342 Acc: 0.0886

Epoch 5/14
----------
train Loss: 1.0183 Acc: 0.7753
test Loss: 3.5628 Acc: 0.2013

Epoch 6/14
----------
train Loss: 0.8989 Acc: 0.8029
test Loss: 3.0634 Acc: 0.3100

Epoch 7/14
----------
train Loss: 0.7642 Acc: 0.8414
test Loss: 2.5533 Acc: 0.4049

Epoch 8/14
----------
train Loss: 0.6235 Acc: 0.8738
test Loss: 2.0448 Acc: 0.5263

Epoch 9/14
----------
train Loss: 0.4918 Acc: 0.9049
test Loss: 1.6369 Acc: 0.6113

Epoch 10/14
----------
train Loss: 0.3827 Acc: 0.9286
test Loss: 1.2734 Acc: 0.6964

Epoch 11/14
----------
train Loss: 0.2845 Acc: 0.9564
test Loss: 1.0327 Acc

In [17]:
model = start_train(models.inception_v3(pretrained=True), aircraft_class, airdata_train, airdata_test, "inception", epochs=20)

Epoch 0/19
----------
train Loss: 4.6322 Acc: 0.0093
test Loss: 4.5320 Acc: 0.0438

Epoch 1/19
----------
train Loss: 4.4808 Acc: 0.0498
test Loss: 4.3784 Acc: 0.1314

Epoch 2/19
----------
train Loss: 4.2253 Acc: 0.1170
test Loss: 4.0226 Acc: 0.1833

Epoch 3/19
----------
train Loss: 3.8288 Acc: 0.1941
test Loss: 3.6120 Acc: 0.2481

Epoch 4/19
----------
train Loss: 3.4240 Acc: 0.2588
test Loss: 3.2147 Acc: 0.2820

Epoch 5/19
----------
train Loss: 3.0146 Acc: 0.3248
test Loss: 2.8320 Acc: 0.3654

Epoch 6/19
----------
train Loss: 2.6349 Acc: 0.3815
test Loss: 2.5132 Acc: 0.4080

Epoch 7/19
----------
train Loss: 2.2966 Acc: 0.4604
test Loss: 2.2386 Acc: 0.4524

Epoch 8/19
----------
train Loss: 2.0151 Acc: 0.5171
test Loss: 2.0469 Acc: 0.4647

Epoch 9/19
----------
train Loss: 1.7763 Acc: 0.5741
test Loss: 1.8456 Acc: 0.4887

Epoch 10/19
----------
train Loss: 1.5695 Acc: 0.6149
test Loss: 1.7503 Acc: 0.4983

Epoch 11/19
----------
train Loss: 1.3862 Acc: 0.6497
test Loss: 1.6270 Acc

In [19]:
model = start_train(models.inception_v3(pretrained=True), aircraft_class, airdata_train, airdata_test, "inception", BackData=back_rgb_inc, back_info=True, epochs=20)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/19
----------
train Loss: 2.6037 Acc: 0.4536
test Loss: 4.6118 Acc: 0.0000

Epoch 1/19
----------
train Loss: 2.4626 Acc: 0.4666
test Loss: 4.5662 Acc: 0.0060

Epoch 2/19
----------
train Loss: 2.4197 Acc: 0.4746
test Loss: 4.4022 Acc: 0.0438

Epoch 3/19
----------
train Loss: 2.2980 Acc: 0.4969
test Loss: 4.0703 Acc: 0.0918

Epoch 4/19
----------
train Loss: 2.1413 Acc: 0.5156
test Loss: 3.7149 Acc: 0.1515

Epoch 5/19
----------
train Loss: 1.9286 Acc: 0.5519
test Loss: 3.2517 Acc: 0.2217

Epoch 6/19
----------
train Loss: 1.7121 Acc: 0.5844
test Loss: 2.8350 Acc: 0.2922

Epoch 7/19
----------
train Loss: 1.4987 Acc: 0.6191
test Loss: 2.4470 Acc: 0.3624

Epoch 8/19
----------
train Loss: 1.3039 Acc: 0.6590
test Loss: 2.1819 Acc: 0.4032

Epoch 9/19
----------
train Loss: 1.1508 Acc: 0.6885
test Loss: 1.9475 Acc: 0.4479

Epoch 10/19
----------
train Loss: 1.0112 Acc: 0.7217
test Loss: 1.7252 Acc: 0.4875

Epoch 11/19
----------
train Loss: 0.9054 Acc: 0.7490
test Loss: 1.6838 Acc

In [23]:
model = start_train(models.resnet18(pretrained=True), kmi_class, kmi_train, kmi_test, "kmni", lrate=0.1)

Epoch 0/14
----------
train Loss: 0.9814 Acc: 0.7165
test Loss: 0.7555 Acc: 0.7742

Epoch 1/14
----------
train Loss: 0.2655 Acc: 0.9161
test Loss: 0.4803 Acc: 0.8608

Epoch 2/14
----------
train Loss: 0.1698 Acc: 0.9466
test Loss: 0.2937 Acc: 0.9157

Epoch 3/14
----------
train Loss: 0.1233 Acc: 0.9613
test Loss: 0.2304 Acc: 0.9295

Epoch 4/14
----------
train Loss: 0.1021 Acc: 0.9682
test Loss: 0.2457 Acc: 0.9258

Epoch 5/14
----------
train Loss: 0.0832 Acc: 0.9742
test Loss: 0.2439 Acc: 0.9323

Epoch 6/14
----------
train Loss: 0.0711 Acc: 0.9775
test Loss: 0.1697 Acc: 0.9523

Epoch 7/14
----------
train Loss: 0.0623 Acc: 0.9799
test Loss: 0.1683 Acc: 0.9556

Epoch 8/14
----------
train Loss: 0.0546 Acc: 0.9826
test Loss: 0.1483 Acc: 0.9576

Epoch 9/14
----------
train Loss: 0.0453 Acc: 0.9854
test Loss: 0.1615 Acc: 0.9569

Epoch 10/14
----------
train Loss: 0.0264 Acc: 0.9919
test Loss: 0.1228 Acc: 0.9676

Epoch 11/14
----------
train Loss: 0.0224 Acc: 0.9934
test Loss: 0.1189 Acc

In [45]:
model = start_train(models.resnet18(pretrained=True), kmi_class, kmi_train, kmi_test, "kmni", BackData = back_gray_kmi, back_info = True, lrate=0.1)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/14
----------
train Loss: 1.1062 Acc: 0.6873
test Loss: 0.7683 Acc: 0.7782

Epoch 1/14
----------
train Loss: 0.2691 Acc: 0.9165
test Loss: 0.3399 Acc: 0.8939

Epoch 2/14
----------
train Loss: 0.1638 Acc: 0.9491
test Loss: 0.2553 Acc: 0.9230

Epoch 3/14
----------
train Loss: 0.1186 Acc: 0.9634
test Loss: 0.2349 Acc: 0.9284

Epoch 4/14
----------
train Loss: 0.0956 Acc: 0.9698
test Loss: 0.2204 Acc: 0.9395

Epoch 5/14
----------
train Loss: 0.0767 Acc: 0.9760
test Loss: 0.1782 Acc: 0.9526

Epoch 6/14
----------
train Loss: 0.0676 Acc: 0.9794
test Loss: 0.1823 Acc: 0.9497

Epoch 7/14
----------
train Loss: 0.0565 Acc: 0.9822
test Loss: 0.1893 Acc: 0.9478

Epoch 8/14
----------
train Loss: 0.0487 Acc: 0.9847
test Loss: 0.1498 Acc: 0.9602

Epoch 9/14
----------
train Loss: 0.0443 Acc: 0.9859
test Loss: 0.1541 Acc: 0.9598

Epoch 10/14
----------
train Loss: 0.0266 Acc: 0.9917
test Loss: 0.1170 Acc: 0.9694

Epoch 11/14
----------
train Loss: 0.0190 Acc: 0.9940
test Loss: 0.1176 Acc

In [48]:
print("EMINST acc validation without background class\n")
model = start_train(models.resnet18(pretrained=True), emi_bal_class, emi_bal_train, emi_bal_test, "emni", lrate=0.1)
print("\nEMINST acc validation with background class\n")
model = start_train(models.resnet18(pretrained=True), emi_bal_class, emi_bal_train, emi_bal_test, "emni", BackData = back_gray_emi, back_info = True, lrate=0.1)

EMINST acc validation without background class

Epoch 0/14
----------
train Loss: 1.2908 Acc: 0.6262
test Loss: 0.5667 Acc: 0.8045

Epoch 1/14
----------
train Loss: 0.5394 Acc: 0.8101
test Loss: 0.4729 Acc: 0.8356

Epoch 2/14
----------
train Loss: 0.4603 Acc: 0.8345
test Loss: 0.4112 Acc: 0.8487

Epoch 3/14
----------
train Loss: 0.4206 Acc: 0.8473
test Loss: 0.4107 Acc: 0.8536

Epoch 4/14
----------
train Loss: 0.3963 Acc: 0.8548
test Loss: 0.3995 Acc: 0.8578

Epoch 5/14
----------
train Loss: 0.3765 Acc: 0.8616
test Loss: 0.4383 Acc: 0.8445

Epoch 6/14
----------
train Loss: 0.3623 Acc: 0.8652
test Loss: 0.3727 Acc: 0.8662

Epoch 7/14
----------
train Loss: 0.3479 Acc: 0.8705
test Loss: 0.3684 Acc: 0.8677

Epoch 8/14
----------
train Loss: 0.3367 Acc: 0.8742
test Loss: 0.3500 Acc: 0.8791

Epoch 9/14
----------
train Loss: 0.3286 Acc: 0.8761
test Loss: 0.3567 Acc: 0.8694

Epoch 10/14
----------
train Loss: 0.2753 Acc: 0.8954
test Loss: 0.3279 Acc: 0.8839

Epoch 11/14
----------
trai

In [49]:
print("EMINST acc validation without background class\n")
model = start_train(models.resnet18(pretrained=True), emi_byc_class, emi_byc_train, emi_byc_test, "emni", lrate=0.1)
print("\nEMINST acc validation with background class\n")
model = start_train(models.resnet18(pretrained=True), emi_byc_class, emi_byc_train, emi_byc_test, "emni", BackData = back_gray_emi, back_info = True, lrate=0.1)

EMINST acc validation without background class



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/14
----------
train Loss: 0.6666 Acc: 0.7833
test Loss: 0.4324 Acc: 0.8454

Epoch 1/14
----------
train Loss: 0.4449 Acc: 0.8395
test Loss: 0.4011 Acc: 0.8531

Epoch 2/14
----------
train Loss: 0.4161 Acc: 0.8476
test Loss: 0.3906 Acc: 0.8567

Epoch 3/14
----------
train Loss: 0.3992 Acc: 0.8527
test Loss: 0.3840 Acc: 0.8570

Epoch 4/14
----------
train Loss: 0.3876 Acc: 0.8560
test Loss: 0.3757 Acc: 0.8598

Epoch 5/14
----------
train Loss: 0.3793 Acc: 0.8584
test Loss: 0.3727 Acc: 0.8591

Epoch 6/14
----------
train Loss: 0.3721 Acc: 0.8606
test Loss: 0.3704 Acc: 0.8626

Epoch 7/14
----------
train Loss: 0.3662 Acc: 0.8625
test Loss: 0.3723 Acc: 0.8625

Epoch 8/14
----------
train Loss: 0.3611 Acc: 0.8641
test Loss: 0.3651 Acc: 0.8623

Epoch 9/14
----------
train Loss: 0.3558 Acc: 0.8656
test Loss: 0.3627 Acc: 0.8644

Epoch 10/14
----------
train Loss: 0.3282 Acc: 0.8746
test Loss: 0.3447 Acc: 0.8712

Epoch 11/14
----------
train Loss: 0.3216 Acc: 0.8765
test Loss: 0.3408 Acc

KeyboardInterrupt: 

In [10]:
model = start_train(models.resnet18(pretrained=True), emi_byc_class, emi_byc_train, emi_byc_test, "emni", BackData = back_gray_emi, back_info = True, lrate=0.1)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 153MB/s] 


Epoch 0/14
----------
train Loss: 0.7345 Acc: 0.7672
test Loss: 0.4644 Acc: 0.8336

Epoch 1/14
----------
train Loss: 0.4640 Acc: 0.8339
test Loss: 0.4305 Acc: 0.8423

Epoch 2/14
----------
train Loss: 0.4321 Acc: 0.8431
test Loss: 0.4006 Acc: 0.8534

Epoch 3/14
----------
train Loss: 0.4129 Acc: 0.8485
test Loss: 0.3956 Acc: 0.8547

Epoch 4/14
----------
train Loss: 0.4003 Acc: 0.8522
test Loss: 0.3734 Acc: 0.8615

Epoch 5/14
----------
train Loss: 0.3909 Acc: 0.8553
test Loss: 0.3792 Acc: 0.8587

Epoch 6/14
----------
train Loss: 0.3836 Acc: 0.8572
test Loss: 0.3793 Acc: 0.8607

Epoch 7/14
----------
train Loss: 0.3772 Acc: 0.8598
test Loss: 0.3813 Acc: 0.8597

Epoch 8/14
----------
train Loss: 0.3720 Acc: 0.8607
test Loss: 0.3678 Acc: 0.8626

Epoch 9/14
----------
train Loss: 0.3672 Acc: 0.8624
test Loss: 0.3670 Acc: 0.8635

Epoch 10/14
----------
train Loss: 0.3390 Acc: 0.8715
test Loss: 0.3473 Acc: 0.8695

Epoch 11/14
----------
train Loss: 0.3329 Acc: 0.8736
test Loss: 0.3472 Acc